In [78]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import together
from time import sleep
import re
from sklearn.metrics import mean_absolute_error
import pandas as pd

from sklearn.metrics import f1_score

In [11]:
def dprint(s, debug):
    if debug:
        print(s)

In [12]:
# TODO: find your API key here
# https://api.together.xyz/settings/api-keys
YOUR_API_KEY = '5a70c1d03697cbf9d7a2c4595ffa8f892bc25311ee89de4dceda77eefb228a84'
together.api_key = YOUR_API_KEY

def call_together_api(prompt, student_configs, pre_processing, post_processing, model='togethercomputer/llama-2-70b', debug=False):
    prompt = pre_processing(prompt)
    output = together.Complete.create(
    prompt = prompt,
    model = model, 
    **student_configs
    )
    dprint('*****prompt*****', debug)
    dprint(prompt, debug)
    dprint('*****result*****', debug)
    res = output['output']['choices'][0]['text']
    dprint(res, debug)
    dprint('*****output*****', debug)
    labels_only = post_processing(res)
    dprint('POST PROCESSED', debug)
    dprint(labels_only, debug)
    dprint('=========', debug)
    return labels_only

In [13]:
model_names = [
    'togethercomputer/llama-2-7b', #LLaMa-2-7B
    'togethercomputer/llama-2-13b', #LLaMa-2-13B
    'togethercomputer/llama-2-70b', #LLaMa-2-70B
    'togethercomputer/llama-2-70b-chat', #LLaMa-2-70B-Chat
    
]

In [71]:
def get_train_df(topn = 10):
    train_df = pd.read_csv('./data/splits/train.csv')
    return train_df[:topn]


In [14]:
def get_test_set():
    test_df = pd.read_csv('./data/splits/test.csv')
    return test_df
    

In [15]:
def get_eval_df(topn = 5):
    eval_df = pd.read_csv('./data/splits/val.csv')
    return eval_df.sample(topn)
    

## Zero Shot Prompt

In [74]:
prompt_prefix_zs = \
'''Offensive speech focuses on the potentially hurtful effect of the tweet content on a given target. This category of text often contains offensive words such as sarcastic remarks, insults, slanders, and slurs.
Based on above definition, classify the following Hinglish tweet into offensive speech or not. 
Your output should only be either "OFF" for offensive, or "NOT" for not offensive.
'''
prompt_examples_zs = "Input Tweet: "
prompt_suffix_zs = "Output: "


In [75]:
# input_string: '@user tweet'
def your_pre_processing_zs(input_string):
    return re.sub(r"@user","", input_string).strip()

def your_post_processing_zs(output_string):
    # using regular expression to find the first consecutive digits in the returned string
    if output_string.strip().lower()[:3]=='off':
        return 1
    else:
        return 0

In [76]:
def test_range(df, prompt_configs, prompt_prefix, examples, prompt_suffix,
               pre_processing=lambda x:x, post_processing=lambda y:y,
               model='togethercomputer/llama-2-70b', debug=False):
    tweet_idx = []
    answers = []
    model_responses = []
    corrected_model_responses = []
    tweet_txt_list = []
    ap_text = ""
    for idx, row in tqdm(df.iterrows()):
        tweet_idx.append(row['id'])
        fixed_prompt = row['tweet_text'] + "\n"
        tweet_txt_list.append(row['tweet_text'])
        fixed_prompt = pre_processing(fixed_prompt)
        prompt = prompt_prefix + examples + fixed_prompt + prompt_suffix
        answer = row['offense']
        answers.append(answer)
        model_response = call_together_api(prompt, prompt_configs, pre_processing, lambda y:y, model=model, debug=debug)
        corrected_model_response = post_processing(model_response)
        corrected_model_responses.append(corrected_model_response)
        model_responses.append(model_response)
        sleep(1)
    df = pd.DataFrame({'tweet_idx': tweet_idx, 'tweet_text': tweet_txt_list, 'model_responses': model_responses, 'corrected_model_responses':corrected_model_responses, 'offense': answers})
    return df

In [77]:
prompt_config_zs = {'max_tokens': 3,
                'temperature': 0.4,
                'top_k': 50,
                'top_p': 0.7,
                'repetition_penalty': 1,
                'stop': []}

model = 'togethercomputer/llama-2-70b-chat'
print(model)

eval_df = get_eval_df(50)

togethercomputer/llama-2-70b-chat


In [61]:
results_df = test_range(eval_df, prompt_config_zs, prompt_examples_zs, prompt_prefix_zs, prompt_suffix_zs, pre_processing=your_pre_processing_zs, post_processing=your_post_processing_zs, model=model, debug=False)
print(results_df)

50it [01:25,  1.72s/it]

    tweet_idx                                         tweet_text  \
0         632  #Demonetisation A dream of many executed by #M...   
1      178287  @user Aren't honourable Judges aware about pre...   
2       79887  @user Muslims harassing Hindus in UK, USA and ...   
3        1062  @user Where are when 2015 floods  came …..when...   
4       34391  INDIAN ARMY CARRIES OUT LARGE FIELD TRAINING E...   
5       65790  is covid-19 still a thing of have they deemed ...   
6      169445  Cricket gods should honour and bless Sikander ...   
7       26597  @user @user Your tweet wasn't worded properly ...   
8       15249  @user @user @user You're just flat wrong about...   
9       75929  @user @user Yes, but the data says whole hindi...   
10      59974  @user Comparison is lame. Biden reduced defici...   
11     181862  @user @user \nIt's a mockery of the system!\nA...   
12      30342  @user Big salute no sir  \nBig big salute Indi...   
13     213424  @user In 1966, Golwalkar, in his 

In [79]:
sum(results_df['corrected_model_responses']==results_df['offense'].values)

29

In [80]:
f1_score(results_df['offense'], results_df['corrected_model_responses'])

0.15999999999999998

## Few Shot Prompting (In Context Learning)
Useful to fix output format

In [137]:
train_df = get_train_df(40)

In [141]:
def create_example(row):
    line1 = "Input Tweet: " + row['tweet_text'] + "\n"
    label = 'OFF' if row['offense']==1 else 'NOT'
    line2 = "Output: " + label + "\n"
    return line1+line2


In [142]:
prompt_examples_icl = ""
for idx,row in train_df.iterrows():
    ex = create_example(row)
    prompt_examples_icl += ex

In [143]:
prompt_examples_icl = prompt_examples_icl + "Input Tweet: "

In [144]:
prompt_prefix_icl = \
'''Offensive speech focuses on the potentially hurtful effect of the tweet content on a given target. This category of text often contains offensive words such as sarcastic remarks, insults, slanders, and slurs.
Based on above definition, classify the following Hinglish tweet into offensive speech or not. Your output should only be either "OFF" for offensive, or "NOT" for not offensive like in the examples below.
'''



prompt_suffix_icl = "Output: "


In [145]:
results_df = test_range(eval_df, prompt_config_zs, prompt_examples_icl, prompt_prefix_icl, prompt_suffix_icl, pre_processing=your_pre_processing_zs, post_processing=your_post_processing_zs, model=model, debug=False)
print(results_df)

50it [01:37,  1.94s/it]

    tweet_idx                                         tweet_text  \
0         789  Petitioners who have challenged Centre’s 2016 ...   
1      213518  Golwalkar as the most important ideologue of t...   
2       11259  @user @user @user @user breastfeeding and havi...   
3        3965  Initial indications from the third quarter sur...   
4      202203  @user Similar to when Hindus of India cheered ...   
5         332  @user @user @user @user I am all for technical...   
6      168732  @user @user @user Lol rcb v csk is not even ri...   
7      213223  @user @user Tariq tell us ,you are against Gol...   
8       75891  @user यादव को pvt charter plane में उड़ते देख ...   
9       17463  @user Whether the Mianlords apply this #Marita...   
10      71559  @user @user @user Every UC who hires is pained...   
11     185291  @user "tackle the climate emergency"\n\nThere ...   
12     164874  @user 's recent announcement has taken EQUAL P...   
13      75941  @user @user And yet they are not 

In [146]:
f1_score(results_df['offense'], results_df['corrected_model_responses'])

0.4444444444444445

## Chain of Thought Prompting

In [155]:
off_tweets = [
"Ho skta h kuch baudh to teri tarah dogle bhi nhi hote..balki bahut hmare sc ya dalit bhai hmare sath mandiro me jate h khub dharm ko mante hai snatan me viswas krte h..or kuch gaddar meem walo ka gana gate hai..wo acha h lekin jo tum kar rhe ho wo ghtiya h..jai meem bol ab",
"Kab aana hai ? # Gharwapasi",
"Haa...Ab Umpire Bhi IPL khelega Chutiya Sala Fixer."]

In [156]:
justification_off_tweets = ["This tweet is offensive because it is insulting in nature", "This tweet is offensive due to the the context of Ghar Wapasi which involves religious conversions", "This tweet is offensive as it insults the umpire and uses cuss words."]

In [157]:
non_off_tweets = ["I don't understand why ppl take law in hands...agar chor tha toh police complaint hone thee...bekar me moka diya jata h ke dalit exploitation ho raha h...", "I have of hindus crimes data agints dalit u hater.\
Dalit hater community.\
Do u have data to prove me otherwise and provide data for muslim crime against hindu", "#demonetisation  .. mention bigger scam than this 😂😂"]

In [158]:
justification_non_off_tweets = ["This tweet is not offensive as it is against lynching of individuals", "This tweet is not offensive as it just asks for more data", "This tweet is not offensive as it criticizes a government policy which is allowed in a healthy democracy"]

In [163]:
def create_examples(off_tweets, justification_off_tweets, non_off_tweets, justification_non_off_tweets):
    examples = ""
    for i in range(len(off_tweets)):
        line1 = "Input Tweet: " + off_tweets[i] + "\n"
        justification_off = "Justification: " + justification_off_tweets[i] + "\n"
        line2 = "Output: " + 'OFF' + "\n"
        line3 = "Input Tweet: " + non_off_tweets[i] + "\n"
        justification_not = "Justification: " + justification_non_off_tweets[i] + "\n"
        line4 = "Output: " + 'NOT' + "\n"
        set = line1 + justification_off + line2 + line3 + justification_not + line4
        examples+=set
    
    return examples


In [166]:
prompt_examples_cot = create_examples(off_tweets, justification_off_tweets, non_off_tweets, justification_non_off_tweets) + "Input Tweet: "

In [167]:
prompt_prefix_cot = \
'''Offensive speech focuses on the potentially hurtful effect of the tweet content on a given target. This category of text often contains offensive words such as sarcastic remarks, insults, slanders, and slurs.
Based on above definition, classify the following Hinglish tweet into offensive speech or not. Your output should be a only be either "OFF" for offensive, or "NOT" for not offensive along with a justification for your output like in the examples below.
'''



prompt_suffix_cot = "Justification: "

In [173]:
"asdka Output: OFF, Output: ".find("Output")

6

In [195]:
# input_string: '@user tweet'
def your_pre_processing_cot(input_string):
    return re.sub(r"@user","", input_string).strip()

def your_post_processing_cot(output_string):
    # using regular expression to find the first consecutive digits in the returned string
    if output_string.find("not offensive")!=-1:
        return 0
    else:
        return 1

In [188]:
prompt_config_cot = {'max_tokens': 30,
                'temperature': 0.4,
                'top_k': 50,
                'top_p': 0.7,
                'repetition_penalty': 1,
                'stop': []}

model = 'togethercomputer/llama-2-70b-chat'
print(model)

eval_df = get_eval_df(50)

togethercomputer/llama-2-70b-chat


In [189]:
results_df = test_range(eval_df, prompt_config_cot, prompt_examples_cot, prompt_prefix_cot, prompt_suffix_cot, pre_processing=your_pre_processing_cot, post_processing=your_post_processing_cot, model=model, debug=False)
print(results_df)

50it [01:44,  2.08s/it]

    tweet_idx                                         tweet_text  \
0      129735  Speak loudly ✊\n\n#GGTU_से_वैदिक_कोर्स_हटाओ\n#...   
1         297  @user Did BJP draft Article 370. Demonetisatio...   
2      178470  @user Have you done any report on lakhs of und...   
3      160124  UAPA/NSA मुसलमानों के लिए-\nसेडिशन/टाडा मुसलमा...   
4      185207  But I do have one emergency solution that I ca...   
5       34260  @user @user @user @user @user @user @user @use...   
6        7624  @user Joe just said another lie.  Joe lies is ...   
7      120494                        @user @user Ghar wapasi hai   
8      159840  SG makes an intervention: This request of hous...   
9      168975  @user Bhai tu apani neutral Game khel yaar tuk...   
10       1065  @user @user @user How much was your loss, due ...   
11        553  @user  Respected Sir, \nThank you for pleading...   
12     159861  Basant: My problem is, I'll take the court int...   
13      59867                              @user

In [196]:
results_df['corrected_model_responses'] = results_df['model_responses'].apply(your_post_processing_cot)

In [199]:
f1_score(results_df['offense'], results_df['corrected_model_responses'])

0.631578947368421